In [143]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np
from src.loaders import load_data, load_labels
from src.classify import MCM_Classifier
from src.plot import  plot_results
import os
from src.plot import plot_confusion_matrix, plot_label_prob_diff
import torch
from collections import defaultdict

In [4]:
# Customizable environment variables
n_categories = 10  # Number of categories to be classified
n_variables = 121  # Number of variables in the dataset
mcm_filename_format = "train-images-unlabeled-{}_comms.dat"
data_filename_format = "train-images-unlabeled-{}.dat"
data_path = "../INPUT/data/"
communities_path = "../INPUT/MCMs/"
output_path = "../OUTPUT/"

In [133]:
# inp = np.genfromtxt(os.path.join(data_path, data_filename_format.format(1)),dtype=int,delimiter=1)
# res = np.where(inp ==1)
# print(np.argwhere(inp[0]))

# inp = np.flip(inp,axis=1)
# indices = np.argwhere(inp==1)
# indices  = np.split(indices[:,1], np.unique(indices[:, 0], return_index=True)[1][1:])
# [l.sort() for l in indices]
# print(indices)

# test = [0]*121
# for i in range(len(test)):
#     if i in indices[0]:
#         test[i] = 1

# print(np.allclose(np.array(test),inp[0]))

In [104]:
size = 2**121
res = np.zeros(size)

ValueError: Maximum allowed dimension exceeded

In [15]:
import numba as nb

In [28]:

def generate_ops(n_var):
    """Generate all single and pairwise operators."""
    return np.array([i for i in range(2**n_var) if i <0]) # TODO are single spin operators allowed?


In [32]:
from itertools import combinations


def generate_ops(num_bits, order_ops, return_binary=False):
    positions = list(range(num_bits))
    combinations_with_ones = list(combinations(positions, order_ops))
    
    result = []
    for combo in combinations_with_ones:
        binary_string = ['0'] * num_bits
        for position in combo:
            binary_string[position] = '1'
        result.append(int(''.join(binary_string), 2))

    return result


In [138]:
res = np.array(generate_ops(121,1) + generate_ops(121,2))

print(res[:100])

[1329227995784915872903807060280344576
 664613997892457936451903530140172288 332306998946228968225951765070086144
 166153499473114484112975882535043072 83076749736557242056487941267521536
 41538374868278621028243970633760768 20769187434139310514121985316880384
 10384593717069655257060992658440192 5192296858534827628530496329220096
 2596148429267413814265248164610048 1298074214633706907132624082305024
 649037107316853453566312041152512 324518553658426726783156020576256
 162259276829213363391578010288128 81129638414606681695789005144064
 40564819207303340847894502572032 20282409603651670423947251286016
 10141204801825835211973625643008 5070602400912917605986812821504
 2535301200456458802993406410752 1267650600228229401496703205376
 633825300114114700748351602688 316912650057057350374175801344
 158456325028528675187087900672 79228162514264337593543950336
 39614081257132168796771975168 19807040628566084398385987584
 9903520314283042199192993792 4951760157141521099596496896
 247588007857076

In [136]:
# index_representation = list(combinations(list(range(10)), 2)) + list(np.arange(10))
# index_representation # cheapest way, each entry of the tuple is the index of where there are 1s in the binary string

In [147]:
# low n
data = ["011","110","101","011"]
data2 = np.array([int(i[::-1], 2) ^ int('1'*len(i), 2) for i in data])
data2
calc_emp_distr(data2)

defaultdict(int, {1: 0.5, 4: 0.25, 2: 0.25})

In [67]:
pseudo_emp_distr = np.array(np.unique(data2, return_counts =True))
pseudo_emp_distr # not normalized
# since indices are just states that are unrelated, their values only matter in uniquely identifying a state. They indicate no distance.

array([[1, 2, 4],
       [2, 1, 1]])

What i will def need custom:
- FWHT or equivalent
- line search evaluation functions. This has to work with my strange datastructure. Could make it a 2 column array instead of tuples.

In [64]:
# if n=3 have 2^3= 8 possible states
# example data data = ["011","110","101","011"]
# data = array([1, 4, 2, 1]) -> after conversion
# FIXME check if we skip over normalization by division by nr_samples ( len(data) )

n_var = 3
len_data = 2**n_var
indices = [0,1,2,3,4,5,6,7]
values =  [0,2,1,0,1,0,0,0]

# data: array([1, 4, 2, 1]) 



calc_emp_distr(3,data2) # array([0.  , 0.5 , 0.25, 0.  , 0.25, 0.  , 0.  , 0.  ])



array([0.  , 0.5 , 0.25, 0.  , 0.25, 0.  , 0.  , 0.  ])

In [75]:
res = np.unique(data2, return_counts =True) # same as pseudo_emp_distr
pairs = list(zip(res[0],res[1]))
map_emp_distr = {i[0]: i[1] for i in pairs} # for now, the keys are the integer represenation. Later these could also be tuples of the indices of the 1s in the bitstring.
map_emp_distr

{1: 2, 2: 1, 4: 1}

In [137]:
# data = np.loadtxt(os.path.join(data_path, data_filename_format.format(1)),dtype="str")

# data = np.array([int(i[::-1], 2) ^ int('1'*len(i), 2) for i in data])
# data 

# import hashlib

# class SparseArray:
#     def __init__(self):
#         self.data = {}

#     def set(self, index, value):
#         # Use a hash function to map the index to a smaller space
#         hash_index = hashlib.sha256(str(index).encode()).hexdigest()
#         self.data[hash_index] = value

#     def get(self, index):
#         hash_index = hashlib.sha256(str(index).encode()).hexdigest()
#         return self.data.get(hash_index, 0)  # Return 0 if the index is not in the dictionary

# # Usage:
# sparse_array = SparseArray()
# sparse_array.set(2**121 - 1, 1)  # Set the value at index 2**121 - 1 to 1
# print(sparse_array.get(2**121 - 1))  # Get the value at index 2**121 - 1

1


In [148]:
# index_representation = list(combinations(list(range(10)), 2)) + list(np.arange(10))
# map_idx_repr = {i:0 for i in index_representation}
# map_idx_repr

In [ ]:
# convert bitstring to tuple representation
inp = np.loadtxt()

# get tuple : count using np.unique
# set to __distr

In [82]:
from collections import defaultdict


test = defaultdict(int)


0

In [166]:
state = "0001"
int(state, 2)

1

In [188]:
from collections import defaultdict

class Sparse_data_distr():
    def __init__(self, data, counts_onlyFalse):
        self.__distr = defaultdict(int) # default value is 0
        self.calc_distr(data)
        self.nr_variables = len(data[0])
        self.nr_samples = len(data)
        self.keys = self.__distr.keys()
        self.counts_only = counts_only



    def calc_distr(self,data):
        single = 1/self.nr_samples if not self.counts_only else 1
        for sample in data:
            self.__distr[int(sample,2)] += single

    def __getitem__(self, key):
        return self.__distr[key] 
    
class Sparse_model_distr():
    def __init__(self, parameters):
        self.__distr = defaultdict(int) # default value is 0
        self.calc_distr(parameters)
        self.nr_variables = len(data[0])
        self.keys = self.__distr.keys()
        

    def calc_distr(self,data):
        g = np.zeros(2**121)

        # these 2 steps do not work with the current implementation
        g[["list of all spin operators here"]] = "model parameters"
        energy = "FWHT of "(g)

        model_distr = np.exp(energy)
        self.model_distr = model_distr / np.sum(model_distr)
        


    def __getitem__(self, key):
        return self.__distr[key] 

    # probably will need a method to tell the line search what the largest values are in the distribution




In [190]:
data = np.loadtxt(os.path.join(data_path, data_filename_format.format(1)),dtype="str")
empr = Sparse_data_distr(data)
print(empr[55])

0


In [ ]:
n_var = 10
# how often can this state be found in data

# should be hashmap with tuple key of state

